### 객체의 콘텍스트 관리 프로토콜 지원

In [18]:
class Foo():
    def __init__(self):
        pass 
    def __enter__(self):
        print('Foo.__enter__()')
        return 'Hello'
    def __exit__(self, exc_ty, exc_val, tb):
        print('Foo.__exit__()')

In [19]:
f = Foo() 

with f as s:
    print(s)

Foo.__enter__()
Hello
Foo.__exit__()


In [34]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None

    def __enter__(self):
        print('__enter__()')
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock

    def __exit__(self, exc_ty, exc_val, tb):
        print('__exit__()')
        self.sock.close()
        self.sock = None

In [35]:
if __name__ == '__main__':
    from functools import partial

    c = LazyConnection(('www.python.org', 80))
    with c as s:
        s.send(b'GET /index.html HTTP/1.0\r\n')
        s.send(b'Host: www.python.org\r\n')
        s.send(b'\r\n')
        resp = b''.join(iter(partial(s.recv, 8192), b''))

    print('Got %d bytes' % len(resp))


__enter__()
__exit__()
Got 392 bytes


### 인스턴스를 많이 생성할 때 메모리 절약

In [45]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

d1 = Date(2020, 6, 1)
print(d1.year, d1.month, d1.day)
# print(dir(d1))
print(d1.__dict__)
d2 = Date(2021, 7, 22)
print(d2.__dict__)
dir(d2)

2020 6 1
{'year': 2020, 'month': 6, 'day': 1}
{'year': 2021, 'month': 7, 'day': 22}


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'day',
 'month',
 'year']

In [60]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

d1 = Date(2020, 6, 1)
print(dir(d1))
print(type(d1.__slots__))
print(d1.__getattribute__(d1.__slots__[0]))
# print(d1.year, d1.month, d1.day)

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'day', 'month', 'year']
<class 'list'>
2020


### 관리 속성 만들기

In [70]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        print('Person.first_name()')
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

In [72]:
class Person:
    def __init__(self, joomin):
        self.joomin = joomin

    def joomin_check(self, joomin):
        seq = "234567892345"
        checksum = 0
        for x, y in zip(joomin, seq):
            checksum += int(x)*int(y)
            
        checksum = (11-checksum%11)%10
        return checksum == int(joomin[12])
        
    @property
    def joomin(self):
        return self._joomin

    @joomin.setter
    def joomin(self, value):
        print('Person.joomin()')
        if not self.joomin_check(value):
            raise ValueError('주민번호가 잘못되어 있습니다.')
        self._joomin = value

In [78]:
p1 = Person("8812202469125")
p1.joomin

Person.joomin()


'8812202469125'

## 상속

### 부모 클래스의 메소드 호출

In [80]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()

if __name__ == '__main__':
    b = B()
    b.spam()

B.spam
A.spam


In [83]:
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

if __name__ == '__main__':
    b = B()
    print(b.x, b.y)

0 1


In [98]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, name):
        print('Proxy.__getattr__(%s)'%(name))
        return getattr(self._obj, name) # getattr(a,  'spam')

    def __setattr__(self, name, value): 
#         print(name, value)
        print('Proxy.__setattr__(%s, %s)'%(name, value))
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            setattr(self._obj, name, value) # setattr(a, 'x', '37')

In [101]:
if __name__ == '__main__':
    class A:
        def __init__(self, x):
            self.x = x
        def spam(self):
            print('A.spam')

    a = A(42)
    p = Proxy(a)
    print(p.x)
    print(p.spam())
    p.x = 37
    print('Should be 37:', p.x)
    print('Should be 37:', a.x)

Proxy.__setattr__(_obj, <__main__.A object at 0x000001F0DD767310>)
Proxy.__getattr__(x)
42
Proxy.__getattr__(spam)
A.spam
None
Proxy.__setattr__(x, 37)
Proxy.__getattr__(x)
Should be 37: 37
Should be 37: 37


### 다이아몬드 형태의 상속 구조의 문제점

In [111]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')

class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

if __name__ == '__main__':
    print(C.__mro__)
    c = C()

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)
Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


In [115]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        super().foo()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
        
    def foo(self):
        print('B.foo()')

class C(A,B):
    def __init__(self):
        super().__init__()
        print('C.__init__')

if __name__ == '__main__':
    print(C.__mro__)
    c = C()

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)
Base.__init__
B.__init__
B.foo()
A.__init__
C.__init__


### 추상클래스

In [117]:
class Car():
    def run(self):
        print("자동차가 달린다.")

class Sonata(Car):
    pass
        
s = Sonata()
s.run()

자동차가 달린다.


In [118]:
class Car():
    def run(self):
        print("자동차가 달린다.")

class Sonata(Car):
    def run(self):
        print("소나타가 달린다.")
        
s = Sonata()
s.run()

소나타가 달린다.


In [121]:
class Car():
    def run(self):
        print(" 달린다.")

class Sonata(Car):
    def run(self):
        print("소나타가" , end="")
        super().run()
        
s = Sonata()
s.run()

소나타가 달린다.


In [127]:
from abc import ABCMeta, abstractmethod

class Car(metaclass=ABCMeta):
    @abstractmethod
    def run(self): 
        pass

class Sonata(Car):
    def run(self):
        print("소나타가 달린다.")

# c = Car()
s = Sonata()
s.run()

소나타가 달린다.


In [129]:
from abc import ABCMeta, abstractmethod

class Car(metaclass=ABCMeta):
    @abstractmethod
    def run(self): 
        pass

class Sonata(Car):
    def run(self):
        print("소나타가 달린다.")
        
class ModelY(Car):
    def run(self):
        print("모델 Y가 달린다.")


car = [Sonata(), ModelY()]
for c in car:
    c.run()

소나타가 달린다.
모델 Y가 달린다.


In [132]:
from abc import ABCMeta, abstractmethod

class Car(metaclass=ABCMeta):
    @abstractmethod
    def run(self): 
        pass
    
    @abstractmethod
    def stop(self): 
        pass

class Sonata(Car):
    def run(self):
        print("소나타가 달린다.")
        
    def stop(self):
        print("소나타가 멈춘다.")
        
class ModelY(Car):
    def run(self):
        print("모델 Y가 달린다.")
        
    def stop(self):
        print("모델 Y가 멈춘다.")


car = [Sonata(), ModelY()]
for c in car:
    c.run()
    
for c in car:
    c.stop()

소나타가 달린다.
모델 Y가 달린다.
소나타가 멈춘다.
모델 Y가 멈춘다.
